In [10]:
import pandas as pd
from ta.trend import MACD
from ta.momentum import RSIIndicator
from sqlalchemy import create_engine, Column, Integer, String, Date, Numeric
from sqlalchemy.orm import declarative_base, sessionmaker
from datetime import date
import functools

# ------------------------------------------
# 1. DATABASE SETUP (SQLAlchemy + SQLite)
# ------------------------------------------

# Create engine to SQLite file database
engine = create_engine('sqlite:///metal_prices.db')

# Base class for model definition
Base = declarative_base()

# IMPORTANT EXPLANATION:
# -----------------------------------------------------
# In Question 2, the table was defined with a schema to store raw metal prices:
# (Date, Metal, Price). In Question 3, we are instructed to **reuse the same structure**
# to now store technical indicators (like MACD and RSI).
#
# Instead of modifying the original schema to add multiple indicator columns (e.g., MACD, RSI),
# we extend it minimally — replacing 'Price' with two new columns:
#   - 'Indicator': stores the name of the metric (e.g., 'MACD' or 'RSI')
#   - 'Value': the computed value of that metric
#
# This ensures:
# 1. Schema consistency with the original structure
# 2. Flexibility to store **any** number of indicators without changing the schema again
# 3. Each row remains atomic: one value per metal, per date, per indicator
# -----------------------------------------------------

class MetalPrice(Base):
    __tablename__ = 'metalprices'

    Id = Column(Integer, primary_key=True, autoincrement=True)
    Date = Column(Date, nullable=False)
    Metal = Column(String(50), nullable=False)
    Indicator = Column(String(50), nullable=False)  # e.g., 'MACD', 'RSI'
    Value = Column(Numeric(18, 4), nullable=False)   # corresponding numeric value

    def __repr__(self):
        return (f"<MetalPrice(Id={self.Id}, Date={self.Date}, "
                f"Metal='{self.Metal}', Indicator='{self.Indicator}', "
                f"Value={self.Value})>")

# Create the table in the database
Base.metadata.create_all(engine)

# Start a DB session
Session = sessionmaker(bind=engine)
session = Session()

# ------------------------------------------
# 2. LOGGING DECORATOR
# ------------------------------------------

# This decorator logs each insert for transparency
def log_execution(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(f"Inserting {args[2]} for {args[1]} on {args[0]}: {args[3]}")
        return func(*args, **kwargs)
    return wrapper

# This function inserts a single indicator value into the database
@log_execution
def insert_indicator_value(date_, metal, indicator, value):
    record = MetalPrice(
        Date=date_,
        Metal=metal,
        Indicator=indicator,
        Value=value
    )
    session.add(record)
    session.commit()

# ------------------------------------------
# 3. LOAD AND FILTER MARKET DATA
# ------------------------------------------

# Load the market CSV, skipping metadata rows
df = pd.read_csv('../data/MarketData.csv', skiprows=6)

# Rename columns to standard names
df.rename(columns={
    'Dates': 'Date',
    'PX_SETTLE': 'Copper',
    'PX_SETTLE.2': 'Zinc'
}, inplace=True)

# Keep only required columns and convert date format
df = df[['Date', 'Copper', 'Zinc']]
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Filter only 2020 and 2021 data
df = df[df['Date'].dt.year.isin([2020, 2021])].copy()

# ------------------------------------------
# 4. COMPUTE INDICATORS AND INSERT TO DB
# ------------------------------------------

def compute_and_insert(df, metal):
    """
    Computes MACD and RSI for a given metal and inserts each
    value into the database **as a separate row**.

    Why separate rows?
    ------------------
    - The table is designed so that each row stores ONE indicator value only.
    - Storing both MACD and RSI in the same row would require a redesign of the schema,
      which goes against the exercise's instructions to reuse the structure from Question 2.
    - Using one row per indicator makes the schema extensible and keeps the logic simple.
    """

    # Prepare clean series
    temp = df[['Date', metal]].dropna().copy()
    temp = temp.rename(columns={metal: 'Price'})

    # Compute MACD and RSI using the 'ta' library
    macd = MACD(close=temp['Price'], window_fast=12, window_slow=26, window_sign=9)
    rsi = RSIIndicator(close=temp['Price'], window=14)

    # Add computed indicators to DataFrame
    temp['MACD'] = macd.macd()
    temp['RSI'] = rsi.rsi()

    # Drop rows where indicators could not be computed (initial window periods)
    temp.dropna(inplace=True)

    # Insert one row for MACD and one for RSI for each date
    for _, row in temp.iterrows():
        insert_indicator_value(row['Date'].date(), metal, 'MACD', round(row['MACD'], 4))
        insert_indicator_value(row['Date'].date(), metal, 'RSI', round(row['RSI'], 4))

# Run this process for both Copper and Zinc
compute_and_insert(df, 'Copper')
compute_and_insert(df, 'Zinc')



Inserting MACD for Copper on 2020-02-05: -156.3104
Inserting RSI for Copper on 2020-02-05: 34.098
Inserting MACD for Copper on 2020-02-06: -147.126
Inserting RSI for Copper on 2020-02-06: 35.3924
Inserting MACD for Copper on 2020-02-07: -143.9972
Inserting RSI for Copper on 2020-02-07: 31.6811
Inserting MACD for Copper on 2020-02-10: -139.5857
Inserting RSI for Copper on 2020-02-10: 32.107
Inserting MACD for Copper on 2020-02-11: -128.3166
Inserting RSI for Copper on 2020-02-11: 39.967
Inserting MACD for Copper on 2020-02-12: -116.5095
Inserting RSI for Copper on 2020-02-12: 41.7364
Inserting MACD for Copper on 2020-02-13: -103.8571
Inserting RSI for Copper on 2020-02-13: 44.1618
Inserting MACD for Copper on 2020-02-14: -95.1538
Inserting RSI for Copper on 2020-02-14: 41.9898
Inserting MACD for Copper on 2020-02-17: -83.1025
Inserting RSI for Copper on 2020-02-17: 46.8677
Inserting MACD for Copper on 2020-02-18: -75.8247
Inserting RSI for Copper on 2020-02-18: 43.8871
Inserting MACD fo

In [11]:
import random

print("\n Sample rows:")
records = session.query(MetalPrice).all()
for r in random.sample(records, min(5, len(records))):
    print(r)



 Sample rows:
<MetalPrice(Id=298, Date=2020-08-31, Metal='Copper', Indicator='RSI', Value=61.7407)>
<MetalPrice(Id=1737, Date=2021-07-07, Metal='Zinc', Indicator='MACD', Value=-7.8191)>
<MetalPrice(Id=1766, Date=2021-07-27, Metal='Zinc', Indicator='RSI', Value=53.4432)>
<MetalPrice(Id=1186, Date=2020-06-16, Metal='Zinc', Indicator='RSI', Value=50.4939)>
<MetalPrice(Id=1099, Date=2020-04-16, Metal='Zinc', Indicator='MACD', Value=-11.2943)>
